In [ ]:
import pandas as pd
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
# Load data
raw_data = pd.read_excel('ncaa_data.xlsx')
positions_df = pd.read_excel('players_with_positions.xlsx')

# No renaming needed as 'Player' column already exists in raw_data
required_columns = {'Team Display Name', 'Athlete', 'Date', 'Opponent', 'PTS', 'FGM', 'FGA', 
                        'FTM', 'FTA', 'REB', 'AST', 'STL', 'BLK', 'TO', 'Win', 'Game Score'}
if not required_columns.issubset(raw_data.columns):
        print("Error: The input data is missing one or more required columns.")

# Merge player position info
raw_data = raw_data.merge(positions_df, on=['Athlete', 'Team Display Name'], how='left')
raw_data['Date'] = pd.to_datetime(raw_data['Date'])

# Calculate possessions and ORtg
raw_data['Possessions'] = raw_data['FGA'] - raw_data['OREB'] + raw_data['TO'] + (0.4 * raw_data['FTA'])
raw_data['ORtg'] = (raw_data['PTS'] / raw_data['Possessions']) * 100
raw_data['Game_Score'] = raw_data['Game Score']  # Rename for consistency

# Per-player rolling averages (last 5 games)
player_stats = raw_data.sort_values('Date').copy()
metrics = ['ORtg', 'Game_Score']
for metric in metrics:
    player_stats[f'Last_5_{metric}_avg'] = player_stats.groupby('Athlete')[metric].transform(
        lambda x: x.rolling(window=5, min_periods=1).mean())


In [ ]:
team_stats_df = raw_data.groupby(['Team Display Name', 'Date', 'Opponent']).agg(
    PTS=('PTS', 'sum'),
    FGA=('FGA', 'sum'),
    FTA=('FTA', 'sum'),
    OREB=('OREB', 'sum'),
    TO=('TO', 'sum'),
    Win=('Win', 'first'),
    Game_Score=('Game Score', 'sum')
).reset_index()

team_stats_df['Possessions'] = team_stats_df['FGA'] - team_stats_df['OREB'] + team_stats_df['TO'] + (0.4 * team_stats_df['FTA'])
team_stats_df['ORtg'] = (team_stats_df['PTS'] / team_stats_df['Possessions']) * 100


In [ ]:
team_points = team_stats_df[['Team Display Name', 'Date', 'Opponent', 'PTS']].copy()
team_points.rename(columns={'Team Display Name': 'Opponent', 
                            'Opponent': 'Team Display Name', 
                            'PTS': 'PTS_Allowed'}, inplace=True)

team_stats_df = team_stats_df.merge(team_points, on=['Team Display Name', 'Date', 'Opponent'], how='left')
team_stats_df['DRtg'] = (team_stats_df['PTS_Allowed'] / team_stats_df['Possessions']) * 100

# Rolling averages for each team
for feature in ['ORtg', 'DRtg', 'Game_Score']:
    team_stats_df[f'Last_5_{feature}_avg'] = team_stats_df.groupby('Team Display Name')[feature]\
        .transform(lambda x: x.rolling(window=5, min_periods=1).mean())

In [ ]:
matchup_data = []
for _, row in team_stats_df.iterrows():
    team = row['Team Display Name']
    opponent = row['Opponent']
    date = row['Date']

    # Get team's recent performance before this game
    team_history = team_stats_df[(team_stats_df['Team Display Name'] == team) & 
                              (team_stats_df['Date'] < date)]
    if team_history.empty:
        continue
    team_features = team_history.iloc[-1]

    # Get opponent's recent performance before this game
    opponent_history = team_stats_df[(team_stats_df['Team Display Name'] == opponent) & 
                                 (team_stats_df['Date'] < date)]
    if opponent_history.empty:
        continue
    opponent_features = opponent_history.iloc[-1]

    matchup_data.append({
        'date': date,
        'team_ortg_avg': team_features['Last_5_ORtg_avg'],
        'opp_ortg_avg': opponent_features['Last_5_ORtg_avg'],
        'team_drtg_avg': team_features['Last_5_DRtg_avg'],
        'opp_drtg_avg': opponent_features['Last_5_DRtg_avg'],
        'team_game_score_avg': team_features['Last_5_Game_Score_avg'],
        'opp_game_score_avg': opponent_features['Last_5_Game_Score_avg'],
        'outcome': row['Win']
    })
    
base_models = [
    ('logreg', LogisticRegression(max_iter=1000, C=0.1)),
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=5)),
    ('xgb', XGBClassifier(learning_rate=0.1, max_depth=3))
]

meta_model = LogisticRegression(random_state=42)
ensemble = StackingClassifier(estimators=base_models, final_estimator=meta_model, stack_method='predict_proba')


matchup_df = pd.DataFrame(matchup_data)
X = matchup_df.drop(columns=['date', 'outcome'])
y = matchup_df['outcome']

split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

ensemble.fit(X_train, y_train)

In [ ]:
def get_optimal_lineup(team_name, opponent_name):
    
    team_players = player_stats[
        (player_stats['Team Display Name'] == team_name)
    ]
    
    latest_stats = team_players.groupby('Athlete').tail(1)
    
    # Drop players with missing position or metrics
    latest_stats = latest_stats.dropna(subset=['Position', 'Last_5_ORtg_avg', 'Last_5_Game_Score_avg'])
    
    # Group players by position
    pos_groups = {
        pos: group for pos, group in latest_stats.groupby('Position')
        if pos in {'PG', 'SG', 'SF', 'PF', 'C'}
    }
    
    # Ensure all positions have at least one player
    if any(pos not in pos_groups or len(pos_groups[pos]) == 0 for pos in ['PG', 'SG', 'SF', 'PF', 'C']):
        raise ValueError("Insufficient players per position")
    
    opponent_latest_date = team_stats_df[team_stats_df['Team Display Name'] == opponent_name]['Date'].max()
    
    opp_stats = team_stats_df[
        (team_stats_df['Team Display Name'] == opponent_name) &
        (team_stats_df['Date'] <= opponent_latest_date)
    ]
    
    if opp_stats.empty:
        raise ValueError("No opponent data available")
    
    opponent_row = opp_stats.iloc[-1]
    
    # Try all combinations of positions
    best_lineup, best_prob = None, -1
    combinations = itertools.product(
        pos_groups['PG'].iterrows(),
        pos_groups['SG'].iterrows(),
        pos_groups['SF'].iterrows(),
        pos_groups['PF'].iterrows(),
        pos_groups['C'].iterrows()
    )
    
    for lineup in combinations:
        lineup_df = pd.DataFrame([x[1] for x in lineup])
        features = pd.DataFrame([{
            'team_ortg_avg': lineup_df['Last_5_ORtg_avg'].mean(),
            'opp_ortg_avg': opponent_row['Last_5_ORtg_avg'],
            'team_drtg_avg': lineup_df['Last_5_ORtg_avg'].mean(),
            'opp_drtg_avg': opponent_row['Last_5_DRtg_avg'],
            'team_game_score_avg': lineup_df['Last_5_Game_Score_avg'].mean(),
            'opp_game_score_avg': opponent_row['Last_5_Game_Score_avg']
        }])
        
        win_prob = ensemble.predict_proba(features)[0][1]
        if win_prob > best_prob:
            best_prob = win_prob
            best_lineup = lineup_df.copy()
    
    return best_lineup[['Athlete', 'Position']]

In [18]:
team = "Sacred Heart Pioneers"
opponent = "Arizona State Sun Devils"

optimal_lineup = get_optimal_lineup(team, opponent)
print("Optimal Lineup:")
print(optimal_lineup)

Optimal Lineup:
             Athlete Position
28320  Sajada Bonner       PG
28317  Nalyce Dudley       SG
11665     Ann Porter       SF
28323   Faith Pappas       PF
28322    Kelsey Wood        C
